## Importing packages

In [1]:
import time
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import random_split
from torch.utils.data.dataset import Dataset
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split

## Reading and processing data

The data used as an example is an image data stored as a single csv file with each row representing an image. Each image is 48*48 pixel stored as a vector of length 2304. The initial column is the emotion represented to each column. The types of emotions are present. Happy, Sad and fear.

This step includes,
    1. Reading the data file and seperating images and labels
    2. Train test split
    3. Normalization
    4. Creating batch process

Creating category to label encodoing and the inverted mapping

In [2]:
cls2id = {"Happy": 0, "Sad": 1, "Fear": 2}
id2cls = ["Happy", "Sad", "Fear"]

Defining data length, batch size, path of the dataset and the device to use for computation. CPU or GPU

In [3]:
datalen = 10817
BATCHSIZE = 10
PATH = "data/FER2013_csv.csv"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

We are gong to define two ways to process the data. Using the pytorch inbuild dataloader with customization and seperate one from scratch

#### Using pytorch functions

Steps:

1. First we are defining the transformation has to be applied on the images after loading. Inclues normalization. Other transformations can be found <a href="https://pytorch.org/docs/stable/torchvision/transforms.html">here</a>.
    
2. Next step is to define custom dataset reader. To use the code for any other dataset all is required to change the functions defined in the class CustomDatasetFromCSV.



In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(0, 255)])

class CustomDatasetFromCSV(Dataset):
    def __init__(self, csv_path, transform = None):
        self.data    = pd.read_csv(csv_path)
        ## For onehot encodng
        #self.labels  = pd.get_dummies(self.data['emotion']).values
        ## For label encoding
        self.labels  = (self.data['emotion']).values
        ## End of encoding
        print("Number of datapoints", len(self.labels))
        self.height  = 48
        self.width   = 48
        self.transform = transform

    def __getitem__(self, index):
        label        = cls2id[self.labels[index]]
        pixels       = self.data[self.data.columns[1:]].values[index].astype(np.float)
        pixels       = pixels.reshape(self.height, self.width)
        if self.transform:
            pixels = self.transform(pixels)
        return pixels, label
    
    def __len__(self):
        return len(self.data)
    
dataset = CustomDatasetFromCSV(PATH, transform)

Number of datapoints 10817


Next we split the dataset using pytorch random_split and build the  trainloader and testloader using pytorch DataLoader

Train test split and batch cration

In [5]:
train_length      = round(0.7 * datalen)
test_length       = round(0.3 * datalen)

trainset, testset = random_split(dataset, [train_length, test_length])

trainloader       = torch.utils.data.DataLoader(trainset, batch_size = BATCHSIZE, shuffle = True, num_workers = 0)
testloader        = torch.utils.data.DataLoader(testset , batch_size = BATCHSIZE, shuffle = True, num_workers = 0)

#### Custom functions

Custom function from scratch to process the data is devided into four functions.

1. load_data()    : To load the data and normalize it and also doing the other required steps to create the image.
2. loader()       : Loader calls load_data to load data and then tranform the data to tensor and make encofing of the labels.
3. data_split()   : To split the dataset into rain and test.
4. create_batch() : To create random batch with mentioned size. It returns a file which upon calling works same as dataloader

In [6]:
def load_data(PATH):
    data     = pd.read_csv(PATH)
    labels   = data["emotion"]
    data     = data.drop(["emotion"], axis = 1)
    images   = np.array(data.values).reshape(len(data.values), 48, 48)
    return images, labels
    
def loader(PATH):
    images, labels = load_data(PATH)
    images = torch.tensor(images)
    images = images.view(images.shape[0], -1, images.shape[1], images.shape[2])

    target = []
    for label in labels.values:
        target.append(cls2id[label])
    target = torch.tensor(target)
    
    return images, target

def data_split(X, Y, test_size, shuffle = True):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size, shuffle = shuffle)
    return(X_train, X_test, Y_train, Y_test)

def create_batch(X, Y, batch_size = 1):
    batch_x = [X[i: i + batch_size] for i in range(0, len(X), batch_size)]
    batch_y = [Y[i: i + batch_size] for i in range(0, len(Y), batch_size)] 
    return list(zip(batch_x, batch_y))

In [7]:
images, target = loader(PATH)
train_X, test_X, train_Y, test_Y = data_split(images, target, test_size = 0.3) 
trainloaderCustom = create_batch(train_X, train_Y, batch_size = BATCHSIZE)

## Time comparison

Here we are going to perform a time comparison to run a for loop over the data sing both our function

In [12]:
def test_time(data):
    start = time.time()
    length = len(data)
    pbar = tqdm(data, total = length, ncols = 800)
    for inputs, labels in pbar:
        inputs = inputs.to(device)
        inputs = inputs.type(torch.cuda.FloatTensor)
        labels = labels.to(device)
    end = time.time()
    print("Time taken {} sec".format(end - start))

Inbuild one

In [13]:
test_time(trainloader)


Time taken 465.165155172348 sec


Custom one

In [14]:
test_time(trainloaderCustom)


Time taken 0.19699788093566895 sec


For some unknon reason inbuild one takes almost 3000 time more time than the other one. 